In [29]:
from importlib import reload
import flymovie as fm
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pickle
from importlib import reload
from flymovie.general_functions import stack_normalize_minmax
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
kernel = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220210_21x25x25pixels_100x50x50voxel.pkl')
gfp_intensity=100

## Isolating cluster number

In [46]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datsets/iso_nclusters'
mask = Sim.make_spherical_mask(100,100,100,40)
sim = Sim(mask, res_z=85, res_ij=85)
eroded_coords = sim.get_eroded_coordinates(10)
eroded_coords = list(zip(eroded_coords[0], eroded_coords[1], eroded_coords[2]))

for n_clusters in range(0,150,5):
    for rep in ['left', 'right']:
        sim = Sim(mask, res_z=85, res_ij=85)
        sim.add_kernel(kernel, res_z=100, res_ij=50)
        # Add free population.
        sim.add_n_objects(10_000, gfp_intensity, fluors_per_object=1, size=1, mode='nuc')
        # Add HLB.
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        # Add clusters.
        sim.add_n_objects(n_clusters, gfp_intensity, fluors_per_object=13, 
                size=2)
        # Add noise and convolve.
        sim.add_noise('poisson')
        sim.convolve()
        sim.resize((250,85,85), order=1)
        sim.im = stack_normalize_minmax(sim.im) * 1000
        filename = str(n_clusters) + '.pkl'
        sim.add_noise('gaussian', sigma=27)
        fm.save_pickle(sim.im, os.path.join(outfolder, rep, filename))




In [49]:
im = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datsets/iso_nclusters/right/5.pkl')
fm.viewer(im, 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

## Isolating cluster fluor number

In [55]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datsets/iso_mols_per_cluster'

mask = Sim.make_spherical_mask(100,100,100,40)
sim = Sim(mask, res_z=85, res_ij=85)
eroded_coords = sim.get_eroded_coordinates(10)
eroded_coords = list(zip(eroded_coords[0], eroded_coords[1], eroded_coords[2]))

for fluors_per_cluster in range(30,37,3):
    for rep in ['left', 'right']:
        sim = Sim(mask, res_z=85, res_ij=85)
        sim.add_kernel(kernel, res_z=100, res_ij=50)
        # Add free population.
        sim.add_n_objects(10_000, gfp_intensity, fluors_per_object=1, size=1, mode='nuc')
        # Add HLB.
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        # Add clusters.
        sim.add_n_objects(60, gfp_intensity, fluors_per_object=fluors_per_cluster, 
                size=2)
        # Add noise and convolve.
        sim.add_noise('poisson')
        sim.convolve()
        sim.resize((250,85,85), order=1)
        sim.im = stack_normalize_minmax(sim.im) * 1000
        filename = str(fluors_per_cluster) + '.pkl'
        sim.add_noise('gaussian', sigma=27)
        fm.save_pickle(sim.im, os.path.join(outfolder, rep, filename))




In [56]:
im = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datsets/iso_mols_per_cluster/left/36.pkl')
fm.viewer(im, 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

143